In [1]:
"""
Author: Andres Melendez
Description: This script retrieves and transforms player data from the Fantasy Football Data Pros API for 2019.
"""

import requests
import pandas as pd
import json

def fetch_api_data(url: str) -> pd.DataFrame:
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)
        return df
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    finally:
        print("API data retrieval process complete.")


In [2]:
def flatten_stats(df: pd.DataFrame) -> pd.DataFrame:
    stats_data = df['stats'].apply(pd.Series)  # Flatten the 'stats' column into a DataFrame
    passing_stats = stats_data['passing'].apply(pd.Series)
    rushing_stats = stats_data['rushing'].apply(pd.Series)
    receiving_stats = stats_data['receiving'].apply(pd.Series)

    # Safely extract statistics using .get() to avoid KeyErrors
    df['Passing Yards'] = passing_stats.get('passing_yds')
    df['Rushing Yards'] = rushing_stats.get('rushing_yds')
    df['Receiving Yards'] = receiving_stats.get('receiving_yds')
    df['Passing TDs'] = passing_stats.get('passing_tds')
    df['Rushing TDs'] = rushing_stats.get('rushing_tds')
    df['Receiving TDs'] = receiving_stats.get('receiving_tds')

    return df


In [3]:
def clean_and_transform_data(df: pd.DataFrame) -> pd.DataFrame:
    df = flatten_stats(df)

    # Drop the 'stats' column because it contains nested dictionaries
    df = df.drop(columns=['stats'])

    # Rename columns to be more readable
    df = df.rename(columns={'player_name': 'Player Name', 'position': 'Position', 'team': 'Team', 'games_played': 'Games Played', 'fumbles_lost': 'Fumbles Lost'})
    
    # Fix casing for team and position
    df['Team'] = df['Team'].str.upper()
    df['Position'] = df['Position'].str.upper()

    # Drop duplicates (now safe because 'stats' column with dictionaries is removed)
    df = df.drop_duplicates()

    # Convert relevant columns to numeric
    numeric_cols = ['Passing Yards', 'Passing TDs', 'Rushing Yards', 'Rushing TDs', 'Receiving Yards', 'Receiving TDs', 'Games Played', 'Fumbles Lost']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Add new columns (e.g., Total Yards)
    df['Total Yards'] = df['Passing Yards'].fillna(0) + df['Rushing Yards'].fillna(0) + df['Receiving Yards'].fillna(0)

    return df


In [4]:
def main():
    url = 'https://www.fantasyfootballdatapros.com/api/players/2019/all'
    api_data = fetch_api_data(url)
    if api_data is not None and not api_data.empty:
        transformed_data = clean_and_transform_data(api_data)
        print(transformed_data.head())
    else:
        print("No data available for transformation.")

if __name__ == "__main__":
    main()


API data retrieval process complete.
   Fumbles Lost  Games Played          Player Name Position Team  \
0           0.0          16.0  Christian McCaffrey       RB  CAR   
1           2.0          15.0        Lamar Jackson       QB  BAL   
2           3.0          15.0        Derrick Henry       RB  TEN   
3           2.0          16.0          Aaron Jones       RB  GNB   
4           2.0          16.0      Ezekiel Elliott       RB  DAL   

   Passing Yards  Rushing Yards  Receiving Yards  Passing TDs  Rushing TDs  \
0            0.0         1387.0           1005.0          NaN          NaN   
1         3127.0         1206.0              0.0          NaN          NaN   
2            0.0         1540.0            206.0          NaN          NaN   
3            0.0         1084.0            474.0          NaN          NaN   
4            0.0         1357.0            420.0          NaN          NaN   

   Receiving TDs  Total Yards  
0            NaN       2392.0  
1            NaN     